In [1]:
!nvidia-smi

Fri Apr 14 11:52:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
# import snscrape.modules.twitter as sntwitter

In [4]:
# df

In [3]:
df = pd.read_csv('Training.csv')
df = df.rename({'c_summary': 'summary'}, axis=1)
from sklearn.utils import shuffle
df = shuffle(df)

In [5]:
df_tr = df[0:1500]

In [6]:
df_val=df[1500:1860]

In [7]:
df_ts=df[1860:]

In [14]:
# df_val

In [18]:
!pip install -U transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00


In [8]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
# from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install rouge_score
!pip install bert_score
!pip install datasets

In [9]:
!huggingface-cli login 


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [ ]:
#converting data to str as it required

In [10]:
df_tr['content'] = df_tr['content'].astype(str).to_list()
df_tr['summary'] = df_tr['summary'].astype(str).to_list()
df_tr.dtypes

<ipython-input-10-2d25b2be6cb0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr['content'] = df_tr['content'].astype(str).to_list()
<ipython-input-10-2d25b2be6cb0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr['summary'] = df_tr['summary'].astype(str).to_list()


content    object
summary    object
dtype: object

In [ ]:
#for fedding into model we need data into pyarrow format so conveting into it

In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
dataset = ds.dataset(pa.Table.from_pandas(df_tr).to_batches())

### convert to Huggingface dataset
arr_tr = Dataset(pa.Table.from_pandas(df_tr))

dataset = ds.dataset(pa.Table.from_pandas(df_val).to_batches())

### convert to Huggingface dataset
arr_val = Dataset(pa.Table.from_pandas(df_val))

dataset = ds.dataset(pa.Table.from_pandas(df_ts).to_batches())

### convert to Huggingface dataset
arr_test = Dataset(pa.Table.from_pandas(df_ts))


In [12]:
from bert_score import score
import sys
import pandas as pd
import numpy as np
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.optimization import Adafactor, AdafactorSchedule
import torch
from datasets import load_dataset, Dataset, load_metric
import gc
from torch import nn 

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "transformersbook/pegasus-samsum"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

metric = load_metric("rouge")

def par_Stop(model: nn.Module):
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters():
        par.requires_grad = False

def em_Stop(model):
    """Freeze token embeddings and positional embeddings for BART and PEGASUS, just token embeddings for t5."""
    model_type = model.config.model_type
    par_Stop(model.model.shared)
    for d in [model.model.encoder, model.model.decoder]:
        par_Stop(d.embed_positions)
        par_Stop(d.embed_tokens)

em_Stop(model)

<ipython-input-14-67e7bef79dac>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
#creating token of data

In [15]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['content'] , max_length = 512, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
dataset_tr = arr_tr.map(convert_examples_to_features, batched = True)
dataset_val = arr_val.map(convert_examples_to_features, batched = True)
dataset_ts = arr_test.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#defining param and using batch size of 2

In [19]:
batch_size = 2
args = Seq2SeqTrainingArguments(
    "pegasus-tweet-sumL",
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    save_strategy="epoch",
    predict_with_generate=True,
    #output_dir = '',
    push_to_hub=True,
    push_to_hub_model_id="pegasus-tweet-sumL",
    push_to_hub_token='*******************************************'
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)

def compute_metrics(eval_pred):
  '''
  Function to compute the ROUGE scores inside the training loop.
  '''
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

/usr/local/lib/python3.9/dist-packages/transformers/training_args.py:1469: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case drigger/pegasus-tweet-sumL).
  warnings.warn(


In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset_tr,
    eval_dataset=dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

trainer.train()

Cloning https://huggingface.co/drigger/pegasus-tweet-sumL into local empty directory.
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.202100,1.684833,44.804600,23.424200,36.558600,41.301400,65.086100


TrainOutput(global_step=750, training_loss=2.096643473307292, metrics={'train_runtime': 1170.8063, 'train_samples_per_second': 1.281, 'train_steps_per_second': 0.641, 'total_flos': 1462164932837376.0, 'train_loss': 2.096643473307292, 'epoch': 1.0})

In [21]:
out = trainer.predict(dataset_ts)

generated_summaries = []
for i in range(0, len(dataset_ts["summary"])): 
  generated_summaries.append(tokenizer.decode(out[0][i], skip_special_tokens =  True))
ground_truth = dataset_ts["summary"]
conversation = dataset_ts["content"]


In [ ]:
#rouge score of our model

In [22]:
out.metrics

{'test_loss': 1.4925072193145752,
 'test_rouge1': 48.4559,
 'test_rouge2': 26.3418,
 'test_rougeL': 39.7548,
 'test_rougeLsum': 44.1302,
 'test_gen_len': 64.61,
 'test_runtime': 155.4315,
 'test_samples_per_second': 0.643,
 'test_steps_per_second': 0.322}

In [23]:
P, R, F1 = score(generated_summaries, ground_truth, lang="en", verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 3.37 seconds, 29.69 sentences/sec


In [ ]:
#bet score

In [25]:
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level precision score: {P.mean():.3f}")
print(f"System level recall score: {R.mean():.3f}")

System level F1 score: 0.894
System level precision score: 0.896
System level recall score: 0.893


In [27]:
# generated_summaries[23]

In [ ]:
#pushing code to hugginface

In [28]:
trainer.push_to_hub() 

To https://huggingface.co/drigger/pegasus-tweet-sumL
   d6cbc8e..a355fee  main -> main

   d6cbc8e..a355fee  main -> main

To https://huggingface.co/drigger/pegasus-tweet-sumL
   a355fee..9a65c4a  main -> main

   a355fee..9a65c4a  main -> main



'https://huggingface.co/drigger/pegasus-tweet-sumL/commit/a355fee768ab785ba16753a4545726d674f038e3'